In [1]:
import numpy as np
import string
import nltk
import random
from nltk.stem.porter import PorterStemmer
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from datetime import datetime, date
import tensorflow as tf
import json

2022-10-22 11:12:26.174429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 11:12:31.905211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-22 11:12:31.905247: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-22 11:12:32.425781: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 11:12:45.681184: W tensorflow/stream_executor/platform/de

In [206]:
with open('intents.json','r') as file:
    data = json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hi!',
    'Hello, thanks for visiting',
    'Hi there, Nice to meet you',
    'Hi there, ']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'services',
   'patterns': ['Which are your services?',
    'How can you help me ?',
    'You tell me that',
    'Do you provide chat support services?',
    'What do you sell?',
    'I want to know what are the services you are providing ?',
    'I want to know what kind of services you are provide ?',
    'I am searching for chatbot',
    'I am looking for live chat support services',
 

In [207]:
stemmer = PorterStemmer()

In [208]:
stop_words = ['is','am','the','a','an','be','are','were',]


In [209]:

words = []
y = []
patterns =[]
x = []
classes = []
# getting the classes(tags) output and patterns from 'data'
for intent in data['intents']:
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
# applying tokenizer to convert the sentences into a list of words        
        tokens = nltk.word_tokenize(pattern)
# here .extend() is used instead of append() since we don't want to append lists in words 
# but its elements i.e words in 'token'
# here 'tokens' is a list of words
        words.extend(tokens)
        patterns.append(pattern)
        y.append(intent['tag'])

# converting to lower case, applyging lemmatization removing the puctuations
# here 'words' is our vocabulary containing all the words 
words = [stemmer.stem(word.lower()) for word  in words if word not in string.punctuation and 
        word not in stop_words]
# converting the list to set to avoid doubling of words in in 'words'
words = sorted(set(words))
words = list(words)

for list_ in patterns:
    list_ = nltk.word_tokenize(list_)
    list_ = [stemmer.stem(lis.lower()) for lis in list_ if lis not in string.punctuation and lis not in stop_words]
    x.append(list_)
        

In [210]:
# applying the one hot encoding on our training data
train_x = []
train_y = []
training = []
empty_list = [0]*len(classes)
for idx,list_ in enumerate(x):
    doc = []
    for word in words:
        doc.append(1) if word in list_ else doc.append(0)
    output = list(empty_list)
    output[classes.index(y[idx])] = 1
    training.append([doc,output])
random.shuffle(training)       
training = np.array(training,dtype = object)
train_x = list(training[:,0])
train_x = np.array(train_x)
train_y = np.array(list(training[:,1]))

In [211]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nTraining stopped at epoch number:",epoch)   
            self.model.stop_training = True
callbacks = myCallback()        


In [212]:
input_shape = (len(train_x[1]),)
output_shape = len(train_y[0])      
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
model.fit(x=train_x, y=train_y, epochs=200, verbose=1,callbacks=[callbacks])

Epoch 1/200
3/3 [==============================] - 0s 3ms/step - loss: 2.6390 - accuracy: 0.0429
Epoch 2/200
3/3 [==============================] - 0s 7ms/step - loss: 2.3388 - accuracy: 0.3000
Epoch 3/200
3/3 [==============================] - 0s 7ms/step - loss: 2.1305 - accuracy: 0.3714
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 1.9351 - accuracy: 0.4000
Epoch 5/200
3/3 [==============================] - 0s 4ms/step - loss: 1.6843 - accuracy: 0.4714
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 1.4103 - accuracy: 0.6857
Epoch 7/200
3/3 [==============================] - 0s 6ms/step - loss: 1.2291 - accuracy: 0.6714
Epoch 8/200
3/3 [==============================] - 0s 7ms/step - loss: 1.1050 - accuracy: 0.7714
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 0.8836 - accuracy: 0.7714
Epoch 10/200
3/3 [==============================] - 0s 6ms/step - loss: 0.7583 - accuracy: 0.8286
Epoch 11/200
3/3 [===========

In [213]:
model.save('test.h5')

In [214]:
# applying bag of words techniques to convert user's text in binary
def bagofwords(msg):
    tokens = nltk.word_tokenize(msg)
    tokens = [stemmer.stem(token.lower()) for token in tokens if token not in string.punctuation and token 
              not in stop_words]
    binary_msg = []
    for word in words:
        binary_msg.append(1) if word in tokens else binary_msg.append(0)
    return np.array(binary_msg)


In [215]:
dic = {
    'phone_dic': ['Thankyou, May I have your phone number ?','Thanks, Can I have your phone number ?'],
    'email_dic': ['Can you share your email address ?','May I ask your email adress ?'],
    'help_dic': ['how can I help you ?'],
    'negative_dic': ['Ok I respect your privacy. ', "That's ok I respect your privacy."]
}

In [216]:
def prediction(msg,tag):
    boleans = [False, False]

    message = bagofwords(msg)
    result = model.predict(np.array([message]))
    result = np.argmax(result,axis=1)
    class_index = list(result)[0] 
    current_tag = classes[class_index]
    if tag == 'greeting_dic':
        if current_tag == 'negative':            
            response1 = random.choice(dic['negative_dic'])
            response2 = random.choice(dic['help_dic'])
            response = response1+response2
            boleans = [True,True]
        elif current_tag == 'name' or current_tag == 'greeting' or current_tag == 'Null' or current_tag == 'affirmative':
            response = random.choice(dic['phone_dic'])
            boleans = [True,True]
            if msg == "":
                boleans[0] = False
             
            
          
    if tag == 'name' or tag == 'affirmative' or tag == 'Null':
        if boleans[1] == False:
            if current_tag == 'negative':
                response1 = random.choice(dic['negative_dic'])
                response2 = random.choice(dic['email_dic'])
                response = response1+' '+response2
                boleans = [True,True]
                
            elif current_tag == 'phone' or current_tag == 'affirmative' or current_tag == 'Null':
                response = 'Thanks, '+ random.choice(dic['email_dic'])
                boleans = [True,True]
                if current_tag == 'affirmative' or current_tag == 'Null':
                    if msg[-1].isnumeric():
                        current_tag = 'phone'
                        
                
    if tag == 'phone' or tag == 'affirmative' or tag == 'negative':
        if current_tag == 'negative':
            response1 = random.choice(dic['negative_dic'])
            response2 = random.choice(dic['help_dic'])
            response = response1 +" "+ response2
            boleans = [True,True]
        elif current_tag == 'affirmative' or current_tag == 'email' or current_tag == 'Null':
            
            response1 = 'Thanks, '
            response2 = random.choice(dic['help_dic'])
            response = response1+response2
            boleans = [True,True]
            if ".com" not in msg:
                print('yes')
                boleans[0] = False
            else:
                current_tag = 'email'
                
    # user se phone number pucha usne mana krdia. ab email pucho
    if tag == 'negative':
        if current_tag == 'negative':
            response1 = random.choice(dic['negative_dic'])
            response2 = random.choice(dic['help_dic'])
            response = response1+response2
            bolean = True
        elif current_tag == 'affirmative' or current_tag == 'email':
            response1 = 'Thankyou, '
            response2 = random.choice(dic['help_dic'])
            response = response1+response2
            bolean = True
        
            
            
    for intent in data['intents']:
        if intent['tag'] == current_tag:
            
            
            if boleans[0] == False:
                response = random.choice(intent['responses'])
            return response,current_tag 

In [217]:
print('Hi! I am chatbot. May I ask your name')
text = input()
response, tag = prediction(text,"greeting_dic") 
while True:
    print(response)
    text = input()
    response,tag =prediction(text,tag)

Hi! I am chatbot. May I ask your name
i am zain
1/1 [==============================] - 0s 53ms/step
Thanks, Can I have your phone number ? --> name
of course, it is +923232612434
1/1 [==============================] - 0s 23ms/step
Thanks, Can you share your email address ? --> phone
zain.18j20000@gmail.com
1/1 [==============================] - 0s 26ms/step
Thanks, how can I help you ? --> email
you tell me how can you help me
1/1 [==============================] - 0s 27ms/step
We are a software house and provide services of live chat support. Are you looking for it? If yes, please share the details of your requirements.  --> services
that's great
1/1 [==============================] - 0s 23ms/step
My pleasure --> thanks
i am looking for chatbot for my website 
1/1 [==============================] - 0s 23ms/step
We are a software house and provide services of live chat support. Are you looking for it? If yes, please share the details of your requirements.  --> services


KeyboardInterrupt: Interrupted by user

In [189]:
email = "zain.18j2000@gmail.com"
if '.com' in email:
    print(5)

5
